# Moku Tools - Phasemeter Module Demo

This notebook demonstrates the basic usage of the `mokutools.phasemeter` module, including:
- Loading phasemeter data from CSV or ZIP files
- Plotting time series data (phase, frequency)
- Calculating power spectral densities
- Plotting spectrum results


In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display

# Import phasemeter module
from mokutools.phasemeter import MokuPhasemeterObject

# Set up matplotlib for better plots
plt.style.use('default')
%matplotlib inline


## 1. Loading Phasemeter Data

The `MokuPhasemeterObject` class is used to load and parse phasemeter data. It can handle:
- CSV files (`.csv`)
- Compressed CSV files (`.csv.zip`)
- MATLAB files (`.mat`) - automatically converted to CSV
- Liquid Instruments binary files (`.li`) - automatically converted to CSV

Let's load a test data file:


In [ ]:
# Load phasemeter data from test file
test_file = Path("../tests/MokuPhasemeterData_MokuProTest.csv.zip")

if not test_file.exists():
    print(f"⚠️  Test file not found: {test_file}")
    print("Please provide a path to your phasemeter data file.")
else:
    # Load the data
    pm = MokuPhasemeterObject(filename=str(test_file))
    
    # Display basic information
    print(f"✅ Data loaded successfully!")
    print(f"   File: {pm.filename}")
    print(f"   Sampling frequency: {pm.fs:.2f} Hz")
    print(f"   Date: {pm.date}")
    print(f"   Number of channels: {pm.nchan}")
    print(f"   Duration: {pm.duration:.2f} seconds")
    print(f"   Number of data points: {len(pm.df)}")
    print(f"\n   Available columns: {list(pm.df.columns)}")


## 2. Inspecting the Data

Let's take a look at the first few rows of the loaded data:


In [ ]:
# Display first few rows
if 'pm' in locals():
    print("First 5 rows of data:")
    display(pm.df.head())
    
    print(f"\nData statistics:")
    display(pm.df.describe())


## 3. Plotting Time Series Data

Let's plot the time series data for phase and frequency measurements:


In [ ]:
if 'pm' in locals():
    # Create figure with subplots
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # Plot phase for all channels
    ax = axes[0]
    for i in range(1, pm.nchan + 1):
        phase_col = f'{i}_phase'
        if phase_col in pm.df.columns:
            ax.plot(pm.df['time'], pm.df[phase_col], label=f'Channel {i}')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Phase (rad)')
    ax.set_title('Phase vs Time')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot frequency for all channels
    ax = axes[1]
    for i in range(1, pm.nchan + 1):
        freq_col = f'{i}_freq'
        if freq_col in pm.df.columns:
            ax.plot(pm.df['time'], pm.df[freq_col], label=f'Channel {i}')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Frequency (Hz)')
    ax.set_title('Frequency vs Time')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


### Plotting a Subset of Data

You can also plot a zoomed-in view of a specific time range:


In [ ]:
if 'pm' in locals():
    # Plot a subset (first 1 second)
    time_limit = 1.0  # seconds
    mask = pm.df['time'] <= time_limit
    
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # Plot phase
    ax = axes[0]
    for i in range(1, pm.nchan + 1):
        phase_col = f'{i}_phase'
        if phase_col in pm.df.columns:
            ax.plot(pm.df.loc[mask, 'time'], pm.df.loc[mask, phase_col], 
                   label=f'Channel {i}', linewidth=1.5)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Phase (rad)')
    ax.set_title(f'Phase vs Time (first {time_limit} s)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot frequency
    ax = axes[1]
    for i in range(1, pm.nchan + 1):
        freq_col = f'{i}_freq'
        if freq_col in pm.df.columns:
            ax.plot(pm.df.loc[mask, 'time'], pm.df.loc[mask, freq_col], 
                   label=f'Channel {i}', linewidth=1.5)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Frequency (Hz)')
    ax.set_title(f'Frequency vs Time (first {time_limit} s)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


## 4. Calculating Power Spectral Densities

The `spectrum()` method computes power spectral density estimates. You can compute spectrums for:
- `'phase'` - Phase noise spectrum
- `'frequency'` - Frequency noise spectrum  
- `'freq2phase'` - Frequency-to-phase converted spectrum

Let's compute phase and frequency spectrums:


In [ ]:
if 'pm' in locals():
    # Compute spectrums for all channels
    print("Computing frequency spectrums...")
    pm.spectrum(['frequency'])
    
    print(f"✅ Spectrums computed!")
    print(f"   Available spectrums: {list(pm.ps.keys())}")


### Computing Spectrums for Specific Channels

You can also compute spectrums for specific channels:


In [ ]:
if 'pm' in locals():
    # Example: Compute phase spectrum for channel 1 only
    # (This will add to existing spectrums, not replace them)
    pm.spectrum('phase', channels=[1, 2])
    
    print(f"Available spectrums: {list(pm.ps.keys())}")


## 5. Plotting Power Spectral Densities

Now let's plot the computed spectrums. The spectrum objects from `speckit` typically have frequency and power spectral density attributes:


In [ ]:
if 'pm' in locals() and len(pm.ps) > 0:
    # Inspect the structure of a spectrum object
    first_key = list(pm.ps.keys())[0]
    spec = pm.ps[first_key]
    print(f"Example spectrum object ({first_key}):")
    print(f"   Type: {type(spec)}")
    print(f"   Attributes: {[attr for attr in dir(spec) if not attr.startswith('_')]}")
    
    # Try to access common attributes
    if hasattr(spec, 'f'):
        print(f"   Frequency array shape: {spec.f.shape}")
    if hasattr(spec, 'psd'):
        print(f"   PSD array shape: {spec.psd.shape}")
    elif hasattr(spec, 'power'):
        print(f"   Power array shape: {spec.power.shape}")
    elif hasattr(spec, 'S'):
        print(f"   S array shape: {spec.S.shape}")


In [ ]:
if 'pm' in locals() and len(pm.ps) > 0:
    # Plot phase spectrums
    phase_keys = [k for k in pm.ps.keys() if 'phase' in k and 'freq2phase' not in k]
    
    if len(phase_keys) > 0:
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        
        for key in phase_keys:
            spec = pm.ps[key]
            # Try different common attribute names for frequency and PSD
            if hasattr(spec, 'f') and hasattr(spec, 'psd'):
                f = spec.f
                psd = spec.psd
            elif hasattr(spec, 'f') and hasattr(spec, 'power'):
                f = spec.f
                psd = spec.power
            elif hasattr(spec, 'f') and hasattr(spec, 'S'):
                f = spec.f
                psd = spec.S
            elif hasattr(spec, 'frequency'):
                f = spec.frequency
                psd = spec.psd if hasattr(spec, 'psd') else spec.power if hasattr(spec, 'power') else spec.S
            else:
                # Fallback: try to access as array-like
                try:
                    f = spec[0] if hasattr(spec, '__getitem__') else None
                    psd = spec[1] if hasattr(spec, '__getitem__') else None
                except:
                    print(f"⚠️  Could not extract frequency/PSD from {key}")
                    continue
            
            if f is not None and psd is not None:
                ax.loglog(f, psd, label=key, linewidth=1.5)
        
        ax.set_xlabel('Frequency (Hz)')
        ax.set_ylabel('Power Spectral Density')
        ax.set_title('Phase Noise Spectrum')
        ax.legend()
        ax.grid(True, alpha=0.3, which='both')
        plt.tight_layout()
        plt.show()
    else:
        print("No phase spectrums found to plot.")


In [ ]:
if 'pm' in locals() and len(pm.ps) > 0:
    # Plot frequency spectrums
    freq_keys = [k for k in pm.ps.keys() if 'freq' in k and 'freq2phase' not in k]
    
    if len(freq_keys) > 0:
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        
        for key in freq_keys:
            spec = pm.ps[key]
            # Try different common attribute names for frequency and PSD
            if hasattr(spec, 'f') and hasattr(spec, 'psd'):
                f = spec.f
                psd = spec.psd
            elif hasattr(spec, 'f') and hasattr(spec, 'power'):
                f = spec.f
                psd = spec.power
            elif hasattr(spec, 'f') and hasattr(spec, 'S'):
                f = spec.f
                psd = spec.S
            elif hasattr(spec, 'frequency'):
                f = spec.frequency
                psd = spec.psd if hasattr(spec, 'psd') else spec.power if hasattr(spec, 'power') else spec.S
            else:
                # Fallback: try to access as array-like
                try:
                    f = spec[0] if hasattr(spec, '__getitem__') else None
                    psd = spec[1] if hasattr(spec, '__getitem__') else None
                except:
                    print(f"⚠️  Could not extract frequency/PSD from {key}")
                    continue
            
            if f is not None and psd is not None:
                ax.loglog(f, psd, label=key, linewidth=1.5)
        
        ax.set_xlabel('Frequency (Hz)')
        ax.set_ylabel('Power Spectral Density')
        ax.set_title('Frequency Noise Spectrum')
        ax.legend()
        ax.grid(True, alpha=0.3, which='both')
        plt.tight_layout()
        plt.show()
    else:
        print("No frequency spectrums found to plot.")


## 6. Loading Data with Time Slicing

You can load a subset of data by specifying `start_time` and `duration`:


In [ ]:
if 'pm' in locals() and test_file.exists():
    # Load a subset of data (starting at 0.5s, duration 2.0s)
    pm_subset = MokuPhasemeterObject(
        filename=str(test_file),
        start_time=0.5,
        duration=2.0
    )
    
    print(f"✅ Subset loaded!")
    print(f"   Start time: {pm_subset.start_time:.2f} s")
    print(f"   Duration: {pm_subset.duration:.2f} s")
    print(f"   Number of data points: {len(pm_subset.df)}")
    
    # Plot the subset
    fig, ax = plt.subplots(1, 1, figsize=(12, 5))
    if '1_phase' in pm_subset.df.columns:
        ax.plot(pm_subset.df['time'], pm_subset.df['1_phase'], linewidth=1.5)
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Phase (rad)')
        ax.set_title(f'Phase vs Time (subset: {pm_subset.start_time:.2f} to {pm_subset.start_time + pm_subset.duration:.2f} s)')
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()


## 7. Precomputing Spectrums During Load

You can also compute spectrums automatically when loading data by passing the `spectrums` parameter:


In [ ]:
if test_file.exists():
    # Load data and compute phase spectrum automatically
    pm_auto = MokuPhasemeterObject(
        filename=str(test_file),
        spectrums=['phase']
    )
    
    print(f"✅ Data loaded with precomputed spectrums!")
    print(f"   Available spectrums: {list(pm_auto.ps.keys())}")


## Summary

This notebook demonstrated the basic usage of the `mokutools.phasemeter` module:

1. **Loading Data**: Use `MokuPhasemeterObject` to load phasemeter data from CSV, ZIP, MAT, or LI files
2. **Time Series Plotting**: Plot phase and frequency measurements vs time
3. **Spectrum Computation**: Use the `spectrum()` method to compute power spectral densities
4. **Spectrum Plotting**: Visualize computed spectrums using log-log plots
5. **Time Slicing**: Load subsets of data using `start_time` and `duration` parameters
6. **Precomputation**: Automatically compute spectrums during data loading

### Key Attributes:
- `pm.df`: Pandas DataFrame containing all time series data
- `pm.fs`: Sampling frequency in Hz
- `pm.nchan`: Number of phasemeter channels
- `pm.ps`: Dictionary of computed power spectral densities
- `pm.date`: Timestamp from the data file

### Common Data Columns:
- `time`: Time array
- `{i}_freq`: Measured frequency for channel i
- `{i}_phase`: Phase in radians for channel i (derived from cycles)
- `{i}_cycles`: Phase in cycles for channel i
- `{i}_set_freq`: Set frequency for channel i
- `{i}_i`, `{i}_q`: I and Q quadrature components for channel i
